In [ ]:
import openai
import os
from langchain.llms import AzureOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
key = os.environ.get("azure_openai_key")
azure_key = os.environ.get("azure_service_principle_key")
openai.api_type = "azure"
openai.api_base = "https://asr-test-env.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = key

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine="testDeployment-Nilesh",
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


In [ ]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
```
"""
response = get_completion(prompt)
print(response)

In [ ]:
print(get_completion("What is the capital of India?"))

In [ ]:
llm = AzureOpenAI(
    openai_api_key=key,
    openai_api_base='https://asr-test-env.openai.azure.com/',
    deployment_name="testDeployment-Nilesh",
    model_name="gpt-35-turbo"
)
llm
llm("Tell me a joke")

In [ ]:
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import MarkdownTextSplitter
markdown_path = "./codeWithUi/A2A_Architecture.md"
loader = UnstructuredMarkdownLoader(markdown_path)
data = loader.load()

In [ ]:
len(data[0].page_content)

In [ ]:
# Create embeddings for each passage for retrieval.
embeddings = OpenAIEmbeddings(
    openai_api_key=key,
    openai_api_base='https://asr-test-env.openai.azure.com/',
    deployment="shashankTextSearch")

In [ ]:
db = Chroma.from_documents(data, embeddings)

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    AzureOpenAI(
    openai_api_key=key,
    openai_api_base='https://asr-test-env.openai.azure.com/',
    deployment_name="testDeployment-Nilesh",
    temperature=0),
    retriever=db,
    memory=memory)


In [ ]:
faq_prompt = """
You are a customer support agent. Customer is asking a question to you. You need to answer it. You only has knowledge that is provided to you via the document A2A_Architecture.md. Do not share the document with the customer.
The question is in triple backticks ```{question}```.
Follow the below steps to answer the question:
0. Forget any other knowledge that you have. Do not use any other questions or answers other than the ones provided in the document.
1. Read the question carefully.
2. Read the document A2A_Architecture.md carefully.
3. If you find the answer to the question in the document, answer the question otherwise go to step 4.
4. Apologise to the customer and ask them to contact the customer support team.

After following the above steps, answer the question.
"""

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain
retriever = db.as_retriever()
chain = RetrievalQAWithSourcesChain.from_chain_type(
    AzureOpenAI(
    openai_api_key=key,
    openai_api_base='https://asr-test-env.openai.azure.com/',
    deployment_name="testDeployment-Nilesh",
    temperature=0),
    chain_type="stuff",
    retriever=retriever,
    max_tokens_limit = 1000
    )
def answerQuestion(question):
    q = faq_prompt.format(question=question) 
    # print(q)
    response = chain({"question": q}, return_only_outputs=True)
    return response


In [ ]:
doc=retriever.get_relevant_documents("ufuvhhjjb india?")

In [ ]:
len(doc[0].page_content)

In [ ]:
answerQuestion("What do you know apart from the source document?")

In [ ]:
answerQuestion("What is recovery resource? No need to share the source with me. Just tell me what you know.")

In [ ]:
answerQuestion("What is failover? How is it different from failback?")

In [ ]:
answerQuestion("What is the Capital of India?")

In [ ]:
answerQuestion("What is the difference between app consistent and crash consistent recovery point?")

In [ ]:
answerQuestion("Which urls should I keep in mind for Azure Site Recovery?")

In [ ]:
answerQuestion("What are rainbow colors?")

In [ ]:
answerQuestion("What is failback??")

In [ ]:
answerQuestion("I have a windows 4012 server. Can I replicate it to Azure?")

In [ ]:
answerQuestion("What all versions of windows are supported?")

In [ ]:
answerQuestion("Can we use powershell with ASR?")

In [ ]:
answerQuestion("Can I use rest api with ASR?")

In [ ]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

In [ ]:
chain = load_qa_with_sources_chain(
    AzureOpenAI(
    openai_api_key=key,
    openai_api_base='https://asr-test-env.openai.azure.com/',
    deployment_name="testDeployment-Nilesh",
    temperature=0),
    chain_type="stuff")
query = "What all versions of windows are supported?"
docs = db.similarity_search(query)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

In [ ]:
# given a vm arm id, fetch the details from azure
from azure.identity import ClientSecretCredential
from azure.mgmt.compute import ComputeManagementClient
azure_credential = ClientSecretCredential(
        client_id="7d595cf7-3da6-4f46-8f37-8d7ba50c520c",
        client_secret=azure_key,
        tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47"
        )

def get_vm_details(vm_arm_id):
    vm_arm_id_arr = vm_arm_id.split('/')
    # Extract the subscription ID, resource group, and VM name from the ARM ID
    subscription_id, resource_group, vm_name = vm_arm_id_arr[2], vm_arm_id_arr[4], vm_arm_id_arr[8]
    print(subscription_id, resource_group, vm_name)
    # Create a ComputeManagementClient instance
    compute_client = ComputeManagementClient(azure_credential, subscription_id)

    # Fetch the VM details using the ARM ID
    vm = compute_client.virtual_machines.get(resource_group, vm_name)

    # Access specific details
    vm_name = vm.name
    vm_location = vm.location
    vm_size = vm.hardware_profile.vm_size
    disks = []
    disks.append({
        'disk_type': 'OS',
        'disk_name': vm.storage_profile.os_disk.name,
        'disk_size': str(vm.storage_profile.os_disk.disk_size_gb),
        'disk_sku': vm.storage_profile.os_disk.managed_disk.storage_account_type
    })
    for data_disk in vm.storage_profile.data_disks:
        disks.append({
            'disk_type': 'Data',
            'disk_name': data_disk.name,
            'disk_size': str(data_disk.disk_size_gb),
            'disk_sku': data_disk.managed_disk.storage_account_type
        })
    # Return the fetched details as a dictionary
    vm_details = {
        'VM Name': vm_name,
        'Location': vm_location,
        'VM Size': vm_size,
        'OS Type': vm.storage_profile.os_disk.os_type,
        'disks': disks
    }
    return vm_details

# get_vm_details for multiple vm_arm_ids that are passed as comma separated string
def get_vm_details_multiple(vm_arm_ids):
    vm_arm_ids_arr = vm_arm_ids.split(',')
    vm_details = []
    for vm_arm_id in vm_arm_ids_arr:
        vm_details.append(get_vm_details(vm_arm_id))
    return vm_details

# get_vm_details for all vms in a resource group
def get_vm_details_rg(resource_group_arm_id):
    resource_group_arm_id_arr = resource_group_arm_id.split('/')
    # Extract the subscription ID and resource group from the ARM ID
    subscription_id, resource_group = resource_group_arm_id_arr[2], resource_group_arm_id_arr[4]
    print(subscription_id, resource_group)
    # Create a ComputeManagementClient instance
    compute_client = ComputeManagementClient(azure_credential, subscription_id)

    # Fetch the VM details using the ARM ID
    vms = compute_client.virtual_machines.list(resource_group)
    vm_details = []
    for vm in vms:
        vm_details.append(get_vm_details(vm.id))
    return vm_details

In [ ]:
# Convert the VM details object to a dictionary
print(get_vm_details("/subscriptions/509099b2-9d2c-4636-b43e-bd5cafb6be69/resourceGroups/PRMYAKA-WUS3-08-03/providers/Microsoft.Compute/virtualMachines/adVM"))
print(get_vm_details_rg("/subscriptions/509099b2-9d2c-4636-b43e-bd5cafb6be69/resourceGroups/PRMYAKA-WUS3-08-03"))


In [ ]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = AzureOpenAI(
    openai_api_key=key,
    openai_api_base='https://asr-test-env.openai.azure.com/',
    deployment_name="testDeployment-Nilesh",
    model_name="gpt-35-turbo"
)

conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)

In [ ]:
system_context_for_pricing = """
Configuring Site Recovery for one Azure Virtual Machine will incur the following charges:
1. Licensing cost: Fixed at $25 per VM per month for windows. $16 per VM per month for Linux.
2. Network Egress cost: Cost to replicate data changes from the disks in source region to the target region. Site Recovery compresses the data by 50% before it is replicated.
3. Storage Cost on the recovery site: Cost of the data stored in the target region. A replica of the source disk is created in the target region with the same size as in source region.
You can assume that the data change rate for a Standard Disk is 10 GB per day and for a Premium Disk is 20 GB per day.
Here is a sample calculation for how to calculate pricing for the below vm:
{'VM Name': 'adVM', 'Location': 'westus3', 'VM Size': 'Standard_D2s_v3', 'OS Type': 'Windows', 'disks': [{'disk_type': 'OS', 'disk_name': 'adVM_OSDisk', 'disk_size': '127', 'disk_sku': 'StandardSSD_LRS'}, {'disk_type': 'Data', 'disk_name': 'adVM_DataDisk', 'disk_size': '20', 'disk_sku': 'StandardSSD_LRS'}, {'disk_type': 'Data', 'disk_name': 'adVM_DataDisk2', 'disk_size': '100', 'disk_sku': 'StandardSSD_LRS'}]}
1. Check the 'OS Type' field, if it is Windows, then the license cost is $25 per month. If it is Linux, then the license cost is $16 per month. In the json above, the OS Type is Windows, hence the license cost is $25 per month.
2. Check the number of disks. Here, in the above json, it has one OS disk and two data disks. Hence, the total number of disks is 3.
3. Check the disk_sku for every disks. If it is StandardSSD_LRS, then the disk is a standard disk. If it is Premium_LRS, then the disk is a premium disk. In the above json, the OS disk and the first data disk are standard disks. The second data disk is a premium disk.
4. Check the disk_size for every disks. In the above json, the OS disk size is 127 GB, the first data disk size is 20 GB and the second data disk size is 100 GB.
3. Storage cost: Customer will pay the price to host the same disks as source region in the target region. In this case, customer will require two S15 disks and three P20 disks. Checking from the pricing data, P20 disk costs $73 and S15 disk costs $12. Hence, three P20 disks and two S15 disks will cost $249.
Hence, customer's total bill will be $25 + $24 + $249 = $293 per month for this VM.
"""
system_input = f"You are assisstant for helping customers for Azure Site recovery. User the info in triple backtics to calculate the pricing for the VMs. ```{system_context_for_pricing}``"

In [ ]:
conversation.predict(input = system_input)

In [ ]:
conversation.predict(input = "What is the license cost if I want to protect 2 VMs with Windows OS, and 1 VM with Linux OS?")

In [ ]:
conversation.predict(input = "Can you provide input in tabular format?")